In [55]:
# Import libraries
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from shapely.geometry import Point

In [2]:
# Import data
df = pd.read_csv('C:/Users/Lumin/OneDrive/Documents/geospatial-data-science/labs/lab5/seattle_house_prices.csv')
df
#coast = gpd.read_file('C:Users/Lumin/OneDrive/Documents/geospatial-data-science/labs/lab5/')

,price,bedrooms,bathrooms,sqft_living,sqft_lot,yr_built,lat,long
0,538000,3,2.25,2570,7242,1951,47.7210,-122.319
1,180000,2,1.00,770,10000,1933,47.7379,-122.233
2,604000,4,3.00,1960,5000,1965,47.5208,-122.393
3,510000,3,2.00,1680,8080,1987,47.6168,-122.045
4,1230000,4,4.50,5420,101930,2001,47.6561,-122.005
...,...,...,...,...,...,...,...,...
19446,475000,3,2.50,1310,1294,2008,47.5773,-122.409
19447,360000,3,2.50,1530,1131,2009,47.6993,-122.346
19448,400000,4,2.50,2310,5813,2014,47.5107,-122.362
19449,400000,3,2.50,1600,2388,2004,47.5345,-122.069


In [3]:
# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['long'], df['lat']))
gdf = gdf.set_crs(4326, allow_override=True)

# Reproject everything to UTM 10N (EPSG:32610)
gdf_utm = gdf.to_crs('EPSG:32610')

In [4]:
# Compute correlation matrix
corr_matrix = gdf_utm.corr()

# Display just house value correlations
corr_matrix["price"].sort_values(ascending= False)

price          1.000000
sqft_living    0.702296
bathrooms      0.524395
bedrooms       0.315804
lat            0.308082
sqft_lot       0.090125
yr_built       0.052453
long           0.020092
Name: price, dtype: float64

### Question 1:

There are 19451 housing units. There are 6 features (When you combine lat and long). There no null values. The highest correlation variables are living area, the number of bathrooms and the number of bedrooms. The least significant variables are the longitude, the year built, and the size of the lot that the unit is on.  

### Question 2:

In [5]:
# Define feature list
feature_list =  ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 
                 'yr_built', 'lat', 'long' ]

# Define features and labels 
X = df[feature_list]
y = df['price']

# Standarize data
scaler = StandardScaler()  
X_scaled = scaler.fit_transform(X)

In [6]:
# Split data 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [10]:
# Define model
forest_reg = RandomForestRegressor(n_estimators = 30)

# Fit model
forest_reg.fit(X_train, y_train)

RandomForestRegressor(n_estimators=30)

In [11]:
# Predict test labels predictions
predictions = forest_reg.predict(X_test)

# Compute mean-squared-error
final_mse = mean_squared_error(y_test , predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

159427.1766550709

In [9]:
# Note to self
# How to create a new variable that are the same distance.  
# gdf_utm["New list name"] = "New list name"

In [19]:
# Import and read coastline data(?), reproject to appropriate system
cl = gpd.read_file('C:/Users/Lumin/OneDrive/Documents/geospatial-data-science/labs/lab5/washington_coastline.shp')
cl_utm = cl.to_crs('EPSG:32610')

I am not particularaly interested in the coastline dataset, I am almost certain that this feature will not have significance when determaining the housing cost. 

In [36]:
gdf = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df['long'], df['lat']))
gdf = gdf.set_crs(4326, allow_override = True)

# Reproject everything to UTM 10N (EPSG:32610)
gdf_utm = gdf.to_crs('EPSG:32610')
cl_utm = cl.to_crs('EPSG:32610')

distance_to_coast = []
for i in range(gdf_utm.shape[0]):
    distance_to_coast.append(cl_utm.distance(gdf_utm['geometry'].iloc[i]).min())
    
# Add to DataFrame
gdf_utm['distance_to_coast'] = distance_to_coast

In [42]:
# plt.scatter(gdf_utm['long'], gdf_utm['lat'], c = gdf_utm['distance_to_coast'])
# plt.colorbar()

In [80]:
df = pd.read_csv('C:/Users/Lumin/OneDrive/Documents/geospatial-data-science/labs/lab5/seattle_house_prices.csv')
gdf = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df['long'], df['lat']))
gdf = gdf.set_crs(4326, allow_override = True)
gdf_utm = gdf.to_crs('EPSG:32610')

# City center location
city_center_true = Point(-122.343506, 47.617004)
dist_cc = gpd.GeoDataFrame(geometry = [city_center_true])
dist_cc = dist_cc.set_crs(4326)
dist_cc_utm = dist_cc.to_crs(32610)

# Calculating distance to the center of the city
distance_to_cc = []
for i in range(gdf_utm.shape[0]):
    distance_to_cc.append(dist_cc_utm.distance(gdf_utm['geometry'].iloc[i]).min())

In [81]:
gdf_utm['city_center'] = distance_to_cc

In [82]:
correlation_matrix = gdf_utm.corr()

correlation_matrix["price"].sort_values(ascending= False)

price          1.000000
sqft_living    0.702296
bathrooms      0.524395
bedrooms       0.315804
lat            0.308082
sqft_lot       0.090125
yr_built       0.052453
long           0.020092
city_center   -0.292956
Name: price, dtype: float64

In [77]:
gdf_utm.columns

Index(['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'yr_built',
       'lat', 'long', 'geometry', 'city_center'],
      dtype='object')

In [83]:
#distance_to_cc

[11703.913328654353,
 15790.855578339879,
 11321.727582630241,
 22432.579365941496,
 25797.655574804357,
 34176.056610544416,
 23161.634555877765,
 11647.166701300106,
 36298.258971340794,
 8987.821602545482,
 17658.42479862626,
 22438.544394220262,
 7005.491402502948,
 5594.255142504292,
 31170.617535954323,
 5312.287822354575,
 35690.29637208131,
 31546.33389708124,
 12616.318780632007,
 34851.563690042654,
 32214.860558758177,
 29948.11440076711,
 35964.512493207134,
 2701.252269422274,
 2288.41442134395,
 9162.248964182227,
 20201.77223340096,
 12311.456226649052,
 7486.654744509316,
 8333.347491406857,
 25303.63866352725,
 7723.057660380418,
 16659.467693419792,
 6073.306496296066,
 31781.482177823997,
 15637.041612766012,
 19118.340196494457,
 25724.31289636372,
 3108.4398213012464,
 22618.442754904067,
 30934.406607264198,
 34560.0402569326,
 14468.734124771228,
 7654.543054030601,
 25008.12806109242,
 17995.85246825261,
 7715.112809622668,
 9301.246341671018,
 6473.505443151454

In [84]:
predictions = forest_reg.predict(X_test)
final_mse = mean_squared_error(y_test, predictions)
final_rmse = np.sqrt(final_mse)

In [85]:
final_rmse

159427.1766550709